# Summarisation: Data cleaning

**Author:** Eva Rombouts  
**Date:** 2024-07-21  
**Version:** 1.0

### Description
This script performs data cleaning on datasets of nursing home notes: ekrombouts/Galaxy_records and ekrombouts/Galaxy_summaries, containing summaries of client notes of fictional nursing home residents. 
The cleaned dataset retains two columns: input and summary. 

In [1]:
# Environment setup
import os

def check_environment():
    try:
        import google.colab
        return "Google Colab"
    except ImportError:
        return "Local Environment"

env = check_environment()

if env == "Google Colab":
    print("Running in Google Colab")
    !pip install -q datasets
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    os.chdir('/content/drive/My Drive/Colab Notebooks/GenCareAI/scripts')
    HF_TOKEN = userdata.get('HF_TOKEN')
else:
    print("Running in Local Environment")
    # !pip install -q
    from dotenv import load_dotenv
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')

Running in Local Environment


In [2]:
import pandas as pd
from datasets import load_dataset

/Users/eva/anaconda3/envs/gcai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("ekrombouts/Galaxy_records", token=HF_TOKEN)
df_records = dataset['train'].to_pandas()
dataset = load_dataset("ekrombouts/Galaxy_summaries", token=HF_TOKEN)
df_summaries = dataset['train'].to_pandas()

Generating train split: 100%|██████████| 3396/3396 [00:00<00:00, 320858.16 examples/s]


In [4]:
df = (
    df_records
    .groupby(['ct_id', 'month', 'iteration'])['note']
    .apply(lambda x: '\n- '.join(x))
    .reset_index()
    .assign(note=lambda df: '- ' + df['note'])
    .merge(df_summaries, on=['ct_id', 'month', 'iteration'], how='right')
    .assign(prev_summary=lambda df: df['summary'].shift(1))
    .dropna(subset=['note'])
    .assign(input=lambda df: df.apply(
        lambda row: f"{row['prev_summary']}\n\nRapportages:\n{row['note']}",
        axis=1
    ))
    [['input', 'summary']]
)

In [6]:
sample = df.sample(2)

for s in sample.iterrows():
    print('---INPUT:---')
    print(s[1]['input'])
    print('\n---SAMENVATTING:---')
    print(s[1]['summary'])
    print('\n*************\n')
    print()

---INPUT:---
Het beloop van meneer Cornelis de Graaf blijft gekenmerkt door aanhoudende uitdagingen met slikproblemen en emotioneel welzijn, waarbij extra ondersteuning wordt geboden. Zorgverleners anticiperen op zijn verminderde mobiliteit, cognitief functioneren en behoeften door individuele benaderingen toe te passen en intensieve zorg te verlenen. Recente aandachtspunten zoals begeleiding bij het eten wegens slikproblemen, rust en ontspanning na inspanning, en het bieden van extra vocht worden gemonitord en opgevolgd in zijn zorgproces.

Rapportages:
- Meneer had vanochtend fysiotherapie en oefende met lopen met begeleiding. Hij maakte goede vorderingen in zijn mobiliteit en kon kortere afstanden afleggen zonder al te veel moeite.
- Tijdens de lunch liep meneer kort rond aan de hand van een verzorger. Hij genoot van zijn maaltijd en had geen slikproblemen.
- Meneer de Graaf had vanavond wat rust in zijn kamer. Hij vertelde een verhaal over vroeger aan een andere bewoner, wat hem on

In [7]:
df.to_csv('../data/galaxy_summaries.csv', index=False)